In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import seaborn as sns

from scipy.stats import pearsonr

sns.set_style("darkgrid")
np.random.seed(930525)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 200)

warnings.simplefilter('once')

%matplotlib inline
%load_ext watermark
%watermark --iversions

matplotlib: 3.3.4
numpy     : 1.20.1
seaborn   : 0.11.1
pandas    : 1.2.2



In [3]:
import autosklearn.classification

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import plot_roc_curve
from sklearn.ensemble import RandomForestClassifier

/home/bhillmann/.conda/envs/automl/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/bhillmann/.conda/envs/automl/lib/python3.8/site-packages/ConfigSpace/__init__.py:37: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ConfigSpace.configuration_space import Configuration, \
/home/bhillmann/.conda/envs/automl/lib/python3.8/site-packages/pyparsing.py:3190: FutureWarnin

In [4]:
import autosklearn.classification

cls = autosklearn.classification.AutoSklearnClassifier()

/home/bhillmann/.conda/envs/automl/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
df_type_1_features = pd.read_csv("../notebooks/strains.dataset.csv")

/home/bhillmann/.conda/envs/automl/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (47,49,55,56,57,58,70,83,95,96,100,103,104,108,109) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
features = ['hits',
 'percent_coverage',
 'mean_coverage',
 'sd_coverage',
 'percent_binned_coverage',
 'mean_binned_coverage',
 'sd_binned_coverage',
 'expected_percent_coverage',
 'shannon_entropy',
 'percent_max_uncovered_region',
 'largest_pileup',
 'largest_binned_pileup',
 'gc_content',
 'total_genome_length',
 'ungapped_genome_length',
 'num_n_groups',
 'consecutive_ns']

In [7]:
X_type_1 = df_type_1_features[features + ["assembly_accession", "dataset", "truth"]]
                                                 
X_type_1 = X_type_1.replace([np.inf, -np.inf], np.nan)
X_type_1 = X_type_1.dropna()

print(X_type_1.groupby('dataset').sum())

y = X_type_1["truth"]
X_type_1 = X_type_1.loc[:, X_type_1.columns.difference(["assembly_accession", "dataset", "truth"])]


X_type_1_train, X_type_1_test, y_train, y_test = train_test_split(X_type_1, y, random_state=42)

                    hits  percent_coverage  mean_coverage    sd_coverage  \
dataset                                                                    
dual_index       1579150         24.456719      77.797533     829.716959   
gis_20         362361709         42.052061   11047.181289  215421.796525   
hmp_even        25248883         44.848096     565.878501   14545.256815   
hmp_staggered   13716590         21.731233     267.918649    9666.614768   
mbarc_26        93698669         32.331114    5516.780499   15708.117984   
zymo_even        9230086         16.898517     434.592579    5085.070763   
zymo_log       125214913         12.027565    3907.630176   64798.277804   

               percent_binned_coverage  mean_binned_coverage  \
dataset                                                        
dual_index                     41.2142              157.9145   
gis_20                         69.4660            36236.1049   
hmp_even                       95.5295             2524.881

In [8]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=60,
    tmp_folder='/tmp/autosklearn_parallel',
    output_folder='../data/autosklearn_parallel',
    n_jobs=20,
    # Each one of the 4 jobs is allocated 3GB
    memory_limit=4000,
    seed=5,
)
automl.fit(X_type_1_train, y_train)

# Print statistics about the auto-sklearn run such as number of
# iterations, number of models failed with a time out.
print(automl.sprint_statistics())

/home/bhillmann/.conda/envs/automl/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/bhillmann/.conda/envs/automl/lib/python3.8/site-packages/autosklearn/automl.py:820: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif X.dtype in (np.float64, np.float):
/home/bhillmann/.conda/envs/automl/lib/python3.8/site-packages/autosklearn/pipeline/base.py:325: DeprecationWarning:

OSError: Timed out trying to connect to inproc://192.168.1.73/4107851/1 after 10 s

In [ ]:
print(automl.sprint_statistics())